In [1]:
from fastai.vision.all import *

Load training data

In [2]:
tensor_list = torch.load( "training_data.pt" )

Load test data

In [3]:
test_list = torch.load("test_data.pt")

Normalize

In [4]:
def normalize(tensor):
    return tensor.float() / 255

tensor_list = list(map(normalize, tensor_list))
test_list = list(map(normalize, test_list))

Baseline

In [5]:
mean_tensors = [tensor.mean(0) for tensor in tensor_list]
len(mean_tensors)

10

In [6]:
def mseMean(mean, tensor):
    return  F.mse_loss(mean, tensor).sqrt()

def mseMeanList(means, tensor):
    return torch.stack( [mseMean(mean,tensor) for mean in means] )

def predict(means, tests, index):
    return torch.argmin( mseMeanList(means, tests[index][0]) ) == index

prediction = [predict(mean_tensors, test_list, i) for i in range(len(test_list))]
prediction

[tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(False),
 tensor(True),
 tensor(True),
 tensor(True),
 tensor(True)]

Simple network

In [90]:
class BasicOptimizer:
    def __init__(self, model, lr):
        self.model = model
        self.lr = lr
           
    def step(self):
        self.model.w1.data = self.model.w1.data - self.lr * self.model.w1.grad.data
        self.model.b1.data = self.model.b1.data - self.lr * self.model.b1.grad.data
        self.model.w2.data = self.model.w2.data - self.lr * self.model.w2.grad.data
        self.model.b2.data = self.model.b2.data - self.lr * self.model.b2.grad.data
    
    def zero_grad(self):
        self.model.w1.grad = None
        self.model.b1.grad = None
        self.model.w2.grad = None
        self.model.b2.grad = None
        
class Model:
    def __init__(self):
        self.w1 = self.init_parameters((28*28,30))
        self.b1 = self.init_parameters(30)
        self.w2 = self.init_parameters((30,10))
        self.b2 = self.init_parameters(10)
        
    def init_parameters(self, size):
        return torch.randn(size).requires_grad_()
        
    def compute(self, x):
        res = x@self.w1 + self.b1
        res = res.max(tensor(0.0))
        res = res@self.w2 + self.b2
        res = torch.nn.functional.softmax( res )
        return res
        
class SimpleNetwork:
    def __init__(self, dl, val_dl, model, optimizer):
        self.dl = dl
        self.val_dl = val_dl
        self.model = model
        self.optimizer = optimizer
    
    def prediction(self, x):
        return self.model.compute(x)
    
    def loss(self, pred, target):
        return torch.nn.functional.cross_entropy(pred, target)
    
    def step(self, x, y):
        pred = self.prediction(x)
        loss = self.loss(pred, y)
        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()
        
    def validate(self, x, y):
        with torch.no_grad():
            pred = self.prediction(x)
            val = torch.eq(
                torch.argmax( pred, dim=1 ),
                torch.argmax( y, dim=1 )
            ).float().mean()
        return val
              
    def learn(self, epochs):
        for epoch in range(epochs):
            print( "epoch " + str(epoch) )
            for xb, yb in self.dl:
                self.step(xb, yb)
                
            accs = [ self.validate(val_xb, val_yb) for val_xb, val_yb in self.val_dl ]
            acc = round( torch.stack( accs ).mean().item(), 4 )
            print("Accuracy " + str(acc))
                

Create training data

In [8]:
def target(size, index):
    tensor = torch.zeros(len(tensor_list))
    tensor[index] = 1
    return tensor

train_x = torch.cat( [tensors for tensors in tensor_list] )
train_y = torch.cat( [torch.stack( [ target(len(tensor_list), i) for j in range(tensor_list[i].size(0)) ]) for i in range(len(tensor_list)) ] )

train_x.shape, train_y.shape

(torch.Size([60000, 784]), torch.Size([60000, 10]))

In [9]:
test_x = torch.cat( [tensors for tensors in test_list] )
test_y = torch.cat( [torch.stack( [ target(len(test_list), i) for j in range(test_list[i].size(0)) ]) for i in range(len(test_list)) ] )

test_x.shape, test_y.shape

(torch.Size([10000, 784]), torch.Size([10000, 10]))

In [10]:
train_y[5], train_y[10000], train_y[50000]

(tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]))

In [13]:
test_y[5], test_y[5000], test_y[8789]

(tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]))

Run network

In [92]:
model = Model()
optimizer = BasicOptimizer(model, 0.001)

dset = list(zip(train_x,train_y))
dl = DataLoader(dset, batch_size=256)

val_dset = list(zip(test_x,test_y))
val_dl = DataLoader(val_dset, batch_size=256)

network = SimpleNetwork(dl, val_dl,model, optimizer)

network.learn(10)

epoch 0


/tmp/ipykernel_469/396899036.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  res = torch.nn.functional.softmax( res )


Accuracy 0.1158
epoch 1
Accuracy 0.116
epoch 2
Accuracy 0.1163
epoch 3
Accuracy 0.1166
epoch 4
Accuracy 0.1168
epoch 5
Accuracy 0.1172
epoch 6
Accuracy 0.1181
epoch 7
Accuracy 0.1186
epoch 8
Accuracy 0.1195
epoch 9
Accuracy 0.1199
